In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# !rm -rf ./gdrive/MyDrive/Song4U/Face_Data/Unscreened_Data/Happy/
# !rm -rf ./gdrive/MyDrive/Song4U/Face_Data/Data/Happy/
# !rm -rf ./gdrive/MyDrive/Song4U/Face_Data/Past_Data/Happy/

# !mkdir ./gdrive/MyDrive/Song4U/Face_Data/Unscreened_Data/Happy/
# !mkdir ./gdrive/MyDrive/Song4U/Face_Data/Data/Happy/
# !mkdir ./gdrive/MyDrive/Song4U/Face_Data/Past_Data/Happy/


In [2]:
#%pip install bs4

import os
from bs4 import BeautifulSoup
import requests as req
import urllib
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np     
from PIL import Image
import sys

#VERY IMPORTANT: Make sure to iterate the number below by 1 before starting the scraping!!!!!
iteration = 50

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [4]:
my_lib_path = os.path.abspath('./gdrive/Shareddrives/RoseHack/')
sys.path.append(my_lib_path)

from landmarks import firstmodify, ifoverborder, finalmodify
net = cv2.dnn.readNetFromCaffe('./gdrive/Shareddrives/RoseHack/deploy.prototxt.txt', './gdrive/Shareddrives/RoseHack/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
key = "Happy"

In [ ]:
def deep_convert(f, return_rectangle = False, save_img=False):
    image = cv2.imread(f)

    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))

    net.setInput(blob)
    detections = net.forward()

    height, width, channels = image.shape
    green = (0, 255, 0)
    line_thick = round(height/120)
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.3:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            len = detections[0, 0, i, 3:7]

            if len[3] < 1:
                (startX, startY, endX, endY) = box.astype("int")
                startX = max(startX, 0)
                startY = max(startY, 0)
                #print(startX, startY)
                left, right, up, bottom = firstmodify(startX, endX, startY, endY)
                #print("1",left, right, up, bottom)
                left, right, up, bottom = ifoverborder(left, right, up, bottom, w, h)
                #print("2",left, right, up, bottom)
                left, right, up, bottom = finalmodify(left, right, up, bottom)
                #print("3",left, right, up, bottom)
                #print(left, right, up, bottom)
                roi = image[up:bottom, left:right]
                #roi = image[startY:endY, startX:endX]
                roi_224 = cv2.resize(roi, (224,224), interpolation = cv2.INTER_AREA)
                head, tail = os.path.split(f)
                
                #cv2.rectangle(image, (left, up), (right, bottom), green, thickness=line_thick)
                output_dir = './gdrive/Shareddrives/RoseHack/Face_Data/Data/' + key + "/" + str(iteration) + f.split("/")[-1]
                print(output_dir)
                cv2.imwrite(output_dir, roi_224)

                #roi = cv2.resize(roi, (200,200), interpolation = cv2.INTER_AREA)
                output = cv2.cvtColor(roi_224, cv2.COLOR_BGR2RGB)
                temp_output = output/255.
                return temp_output, output_dir

In [ ]:
search_words = ["cheerful faces"]
img_dir = "./gdrive/Shareddrives/RoseHack/Face_Data/Unscreened_Data/" + key + "/"
for s in range(len(search_words)):
    #Sources with free photos
    search_words[s] = "unsplash photos of " + search_words[s]
    search_words.append("pexels photos of " + search_words[s])
    search_words.append("pixabay photos of " + search_words[s])
    search_words.append("freeimages photos of " + search_words[s])    

In [ ]:
for word in search_words:
    dir_path = img_dir + word.replace(" ","_")
    urlKeyword = urllib.parse.quote((word))
    url = "https://www.google.com/search?hl=jp&q=" + urlKeyword + "&btnG=Google+Search&tbs=0&safe=off&tbm=isch"
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",}
    r = urllib.request.Request(url=url, headers=headers)
    page = urllib.request.urlopen(r)
    html = page.read().decode("utf-8")
    ht = BeautifulSoup(html, "html.parser")
    elems = ht.findAll("img", {"class":"rg_i Q4LuWd"})
    for li in range(len(elems)):
        #i+=1
        if str(elems[li]).find("https")!=-1:
            if str(elems[li]).find("Unsplash")!=-1 or str(elems[li]).find("Pexels")!=-1 or str(elems[li]).find("Pixabay")!=-1:
                ele = elems[li]
                ele = str(ele).replace('"','').split('src=')
                eledict = dict()
                ele = ele[-1].split(" ")
                imageURL = ele[0]
                file_path = dir_path + str(li) + ".jpg"
                try:
                    urllib.request.urlretrieve(imageURL, file_path)
                    img = Image.open(file_path).convert("RGB")
                    img.save(file_path)
                    if os.path.getsize(file_path)<130000:
                        #print(os.path.getsize(file_path))
                        img2 = Image.open(file_path)
                        new_img = img2.resize((2000,2000))
                        new_img.save(file_path)
                except:
                    "Unable to download " + word + " image " + str(li)
            

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

import shutil
import numpy as np
import random
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
#categories = ["Crime", "Crime_Alert", "Emergency", "Medical", "Medical_Alert", "No_Action", "No_Emergency"]


source = "./gdrive/Shareddrives/RoseHack/Face_Data/Unscreened_Data/" + key + "/"
dest = "./gdrive/Shareddrives/RoseHack/Face_Data/Data/" + key + "/"
past = "./gdrive/Shareddrives/RoseHack/Face_Data/Past_Data/"
files0 = os.listdir(past)
files = os.listdir(source)
img_ref = []
for r in files0:
  img_path = past + r
  im = load_img(img_path, target_size=(224, 224), grayscale=False) 
  img_ref.append(im)
for f in files:
        img_path = source + f
        img = load_img(img_path, target_size=(224, 224), grayscale=False)  # this is a PIL image
        
        #Screening:
        # match = 0
        # for f0 in files0:
        #     img_path0 = fdir0 + f0
        #     img0 = load_img(img_path0, target_size=(224,224), grayscale=False)
        #     if (np.array(img) == np.array(img0)).all():
        #         match+=1

        match = 0
        f0 = 0
        while match==0 and f0<len(img_ref):
            if (np.array(img) == np.array(img_ref[f0])).all():
                match+=1
            f0+=1
        
        if match<1:
            #shutil.copy(img_path, past + f)
            try:
              img00, img_dir00 = deep_convert(img_path)
            except:
              print("Discard")
            shutil.move(img_path, past + f)
            print(dest + f)
        else:
            print("Repeat detected: Discarding image " + img_path)
            os.remove(img_path)

Repeat detected: Discarding image ./gdrive/Shareddrives/RoseHack/Face_Data/Unscreened_Data/Happy/unsplash_photos_of_cheerful_faces20.jpg
Discard
./gdrive/Shareddrives/RoseHack/Face_Data/Data/Happy/unsplash_photos_of_cheerful_faces21.jpg
Repeat detected: Discarding image ./gdrive/Shareddrives/RoseHack/Face_Data/Unscreened_Data/Happy/unsplash_photos_of_cheerful_faces22.jpg
Repeat detected: Discarding image ./gdrive/Shareddrives/RoseHack/Face_Data/Unscreened_Data/Happy/unsplash_photos_of_cheerful_faces23.jpg
Repeat detected: Discarding image ./gdrive/Shareddrives/RoseHack/Face_Data/Unscreened_Data/Happy/unsplash_photos_of_cheerful_faces24.jpg
Repeat detected: Discarding image ./gdrive/Shareddrives/RoseHack/Face_Data/Unscreened_Data/Happy/unsplash_photos_of_cheerful_faces25.jpg
./gdrive/Shareddrives/RoseHack/Face_Data/Data/Happy/50unsplash_photos_of_cheerful_faces26.jpg
./gdrive/Shareddrives/RoseHack/Face_Data/Data/Happy/unsplash_photos_of_cheerful_faces26.jpg
Repeat detected: Discarding i

In [ ]:
!ls ./gdrive/Shareddrives/RoseHack/Face_Data/Data/Neither/ | wc -l

122


In [ ]:
# happy = "./gdrive/Shareddrives/RoseHack/Face_Data/Data/Happy/"
# sad = "./gdrive/Shareddrives/RoseHack/Face_Data/Data/Sad/"
# files0 = os.listdir(happy)
# print(len(files0))
# for f in files0:
#   if f in os.listdir(happy):
#     img_path0 = happy + f
#     img = load_img(img_path0, target_size=(224, 224), grayscale=False) 
#     files = os.listdir(happy)
#     files.remove(f)
#     match = False
#     r = 0
#     print(len(files))
#     while r < len(files) and match==False:
#       img_path = happy + files[r]
#       try:
#         im = load_img(img_path, target_size=(224, 224), grayscale=False)
#         if (np.array(img) == np.array(im)).all():
#           match = True
#           print("Match")
#       except:
#         print("Failed")
#       r+=1
#     if match==True:
#       os.remove(img_path)
    

  # match = 0
  # f0 = 0
  # while match==0 and f0<len(img_ref):
  #     if (np.array(img) == np.array(img_ref[f0])).all():
  #         match+=1
  #     f0+=1
  # if match>=1:



In [ ]:
#img_path

In [ ]:
# dir = "./gdrive/Shareddrives/RoseHack/Face_Data/Past_Data/Neither/"
# dest = "./gdrive/Shareddrives/RoseHack/Face_Data/Past_Data/"
# files = os.listdir(dir)
# for f in files:
#   shutil.move(dir + f, dest + f)